In [ ]:
%pip install omegaconf==2.0.6

In [1]:
%cd FeatureExtraction
%cd video_features

[WinError 2] The system cannot find the file specified: 'FeatureExtraction'
e:\Temporal-Action-Video-Segmentation\FeatureExtraction
e:\Temporal-Action-Video-Segmentation\FeatureExtraction\video_features


In [2]:
from models.i3d.extract_i3d import ExtractI3D
from utils.utils import build_cfg_path
from omegaconf import OmegaConf
import torch

device = 'cuda' if torch.cuda.is_available() else 'cpu'
# torch.cuda.get_device_name(0)

In [ ]:
!pip install torch
!pip install torchvision
!pip install scipy


In [4]:
import os
import cv2
import numpy as np
from tqdm import tqdm

# Select the feature type
feature_type = 'i3d'

# Load and patch the config (assuming you have the correct setup)
args = OmegaConf.load(build_cfg_path(feature_type))
args.device = device
args.flow_type = 'raft'

# Load the model (assuming you have the correct setup)
extractor = ExtractI3D(args)

# Directory containing video files
video_dir = '../../Input/data'  # Change this to your video directory

# Loop through subdirectories in the video directory
for sub_dir in os.listdir(video_dir):
    sub_dir_path = os.path.join(video_dir, sub_dir)

    if os.path.isdir(sub_dir_path):
        # Create a corresponding subdirectory in the extracted features folder
        output_sub_dir = os.path.join('../../Output/features')
        os.makedirs(output_sub_dir, exist_ok=True)

        # Create a folder for ground truth if it doesn't exist
        ground_truth_dir = os.path.join('../../Output/groundTruth')
        os.makedirs(ground_truth_dir, exist_ok=True)

        # Extract features for all video files in the subdirectory
        for video_file in os.listdir(sub_dir_path):
            if video_file.endswith('.mp4') or video_file.endswith('.avi'):
                video_path = os.path.join(sub_dir_path, video_file)
                print(f'Extracting features for {video_file}')

                cap = cv2.VideoCapture(video_path)
                fps = int(cap.get(cv2.CAP_PROP_FPS))
                num_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
                cap.release()
                feature_dict = extractor.extract(video_path)
                for key, value in feature_dict.items():
                    
                    reshaped_feature = np.resize(value, (2048, num_frames))
                    # Create a separate .npy file for each feature type
                    npy_filename = os.path.join(output_sub_dir, f'{video_file.split(".")[0]}_{key}.npy')
                    txt_filename = os.path.join(ground_truth_dir, f'{video_file.split(".")[0]}_{key}.txt')
                    np.save(npy_filename, reshaped_feature)

                    with open(txt_filename, 'w') as txt_file:
                            for _ in range(num_frames):
                                txt_file.write(os.path.splitext(video_file)[0].split('_')[0] + '\n')  # Write sub_dir_path `num_frames` times
                    print(f'Saved {key} feature as {npy_filename}')


                print(f'Processed {video_file}')


Extracting features for v_Biking_g01_c01.avi


d:\Softwares\Extra\Python\Lib\site-packages\torch\functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ..\aten\src\ATen\native\TensorShape.cpp:3484.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


Saved rgb feature as ../../Output/features\v_Biking_g01_c01_rgb.npy
Saved flow feature as ../../Output/features\v_Biking_g01_c01_flow.npy
Saved fps feature as ../../Output/features\v_Biking_g01_c01_fps.npy
Saved timestamps_ms feature as ../../Output/features\v_Biking_g01_c01_timestamps_ms.npy
Processed v_Biking_g01_c01.avi
Extracting features for v_Biking_g01_c02.avi
Saved rgb feature as ../../Output/features\v_Biking_g01_c02_rgb.npy
Saved flow feature as ../../Output/features\v_Biking_g01_c02_flow.npy
Saved fps feature as ../../Output/features\v_Biking_g01_c02_fps.npy
Saved timestamps_ms feature as ../../Output/features\v_Biking_g01_c02_timestamps_ms.npy
Processed v_Biking_g01_c02.avi
Extracting features for v_Biking_g01_c03.avi
Saved rgb feature as ../../Output/features\v_Biking_g01_c03_rgb.npy
Saved flow feature as ../../Output/features\v_Biking_g01_c03_flow.npy
Saved fps feature as ../../Output/features\v_Biking_g01_c03_fps.npy
Saved timestamps_ms feature as ../../Output/features\v

KeyboardInterrupt: 

In [15]:
import os
import random


output_dir = '../../Output/splits'
os.makedirs(output_dir, exist_ok=True)

ground_truth_dir = '../../Output/groundTruth'

# List all ground truth files
ground_truth_files = os.listdir(ground_truth_dir)

# Create dictionaries to store file names for each category
category_files = {
    "_rgb": [],
    "_flow": [],
    "_timestamps_ms": [],
    "_fps": []
}

# Extract the category names and group them by file type (e.g., "_fps", "_flow", etc.)
for filename in ground_truth_files:
    parts = filename.split('_')
    category = f"_{parts[-1].split('.')[0]}"  # Extract the category name from the filename
    if category in category_files:
        category_files[category].append(filename)

# Create split files
for split_num in range(1, 5):
    split_train_file = os.path.join(splits_dir, f'train.split{split_num}.bundle')
    split_test_file = os.path.join(splits_dir, f'test.split{split_num}.bundle')

    train_files = []
    test_files = []

    # Assign specific categories to each split
    if split_num == 1:
        train_categories = ["_rgb"]
    elif split_num == 2:
        train_categories = ["_flow"]
    elif split_num == 3:
        train_categories = ["_timestamps_ms"]
    elif split_num == 4:
        train_categories = ["_fps"]
    
    for category, files in category_files.items():
        # Split each category into 80% train and 20% test based on assigned categories
        if category in train_categories:
            num_files = len(files)
            split_index = int(num_files * 0.8)
            train_files.extend(files[:split_index])
            test_files.extend(files[split_index:])
    
    # Randomly shuffle the file names within each split
    random.shuffle(train_files)
    random.shuffle(test_files)

    # Write the shuffled file names to the split files
    with open(split_train_file, 'w') as train_split_file, open(split_test_file, 'w') as test_split_file:
        train_split_file.write('\n'.join(train_files)+ '\n')
        test_split_file.write('\n'.join(test_files)+ '\n')

print("Splits created successfully.")

Splits created successfully.


In [19]:
# Directory containing ground truth files
ground_truth_dir = '../../Output/groundTruth'  # Change this to your ground truth directory

# Create a folder for splits if it doesn't exist
output_dir = '../../Output'
os.makedirs(output_dir, exist_ok=True)

# List all ground truth files
ground_truth_files = os.listdir(ground_truth_dir)

# Create a dictionary to store category indices
category_indices = {}

# Extract the category names and assign indices
category_index = 0
for filename in ground_truth_files:
    parts = filename.split('_')
    category =parts[0]  # Extract the category name
    if category not in category_indices:
        category_indices[category] = category_index
        category_index += 1

# Create the mapping content
mapping_content = '\n'.join([f'{index} {category}' for category, index in category_indices.items()])

# Write the mapping content to the mapping.txt file
mapping_filename = os.path.join(output_dir, 'mapping.txt')
with open(mapping_filename, 'w') as mapping_file:
    mapping_file.write(mapping_content+'\n')

print(f'Created {mapping_filename} successfully.')


Created ../../Output\mapping.txt successfully.


In [ ]:
!pip freeze